In [64]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from matplotlib import pyplot as plt

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import numpy as np

import json
import glob
import random
import pickle
from tqdm import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch import nn
import torch.nn.functional as F
import spotPython.torch.netcore as netcore
from numpy import meshgrid, array, ravel

from src.eda import EDA
eda = EDA()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
torch.manual_seed(40)
random.seed(40)
np.random.seed(40)

In [65]:
pio.templates.default = "plotly_white"
plotly.offline.init_notebook_mode(connected=True)

In [4]:
class FashionCNN(nn.Module):
    def __init__(self, l1=64):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, l1)
        self.fc2 = nn.Linear(l1, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
class Net_FashionMNIST(netcore.Net_Core):
    def __init__(self, l1, lr_mult, batch_size, epochs, k_folds, patience,
    optimizer, sgd_momentum):
        super(Net_FashionMNIST, self).__init__(
            lr_mult=lr_mult,
            batch_size=batch_size,
            epochs=epochs,
            k_folds=k_folds,
            patience=patience,
            optimizer=optimizer,
            sgd_momentum=sgd_momentum,
        )

        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, l1)
        self.fc2 = nn.Linear(l1, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))])

    trainset = torchvision.datasets.FashionMNIST(data_dir,
        download=True,
        train=True,
        transform=transform)
    testset = torchvision.datasets.FashionMNIST(data_dir,
        download=True,
        train=False,
        transform=transform)

    return trainset, testset

def train_fashion_mnist(config):
    net = FashionCNN(config["config.l1"]) 

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    # # loading data
    trainset, testset = load_data()
    
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=config["config.batch_size"], shuffle=True, num_workers=2
    )

    # defining the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["config.learning_rate"])

    for epoch in range(config["config.epochs"]):
        running_loss = 0.0
        epoch_steps = 0
        with tqdm(enumerate(trainloader, 0), total=len(trainloader), unit="batch") as epoch_iterator:
            for i, data in epoch_iterator:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                epoch_steps += 1

                if i % 1000 == 999:
                    average_loss = running_loss / epoch_steps
                    epoch_iterator.set_description(f"Epoch {epoch+1}, Loss: {average_loss:.3f}")
                    running_loss = 0.0
            

    print("Training finished.")
    return net

def test_accuracy(net):
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

def test_class_accuracy(net):
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct_per_class = [0] * len(class_names)
    total_per_class = [0] * len(class_names)
    correct = 0
    total = 0

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            
            for i in range(len(labels)):
                label = labels[i]
                prediction = predicted[i]
                if label == prediction:
                    correct_per_class[label] += 1
                total_per_class[label] += 1
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy_per_class = [correct / total * 100 if total > 0 else 0 for correct, total in zip(correct_per_class, total_per_class)]

    for i, accuracy in enumerate(accuracy_per_class):
        print('Accuracy of {}: {:.2f}%'.format(class_names[i], accuracy))
        
    accuracy = correct / total
    print('Overall Accuracy: {:.2f}%'.format(accuracy * 100))
    return accuracy_per_class, accuracy

eval_data_path = './evaluation_data/'

def experimentPathToDataframe(experiment_path):
    df_results = pd.DataFrame()
    for file in glob.iglob(experiment_path + '/**/result.json', recursive=True):
        print(f"Loading results from {experiment_path}...")

        df = pd.read_json(file,lines=True)
        data = json.loads(df.to_json(orient='records'))
        df = pd.json_normalize(data, max_level=1)
        df_results = pd.concat([df_results, df], ignore_index=True)

    print(f"----------------------------\nFound {len(df_results.trial_id.unique())} trials.")
    return df_results

def experimentPathToSpotTuner(experiment_path):
    with open(experiment_path, 'rb') as f:
        spot_tuner =  pickle.load(f)
    return spot_tuner

def spotTunerToDataFrame(spotTuner):
    df = pd.DataFrame(np.concatenate((spotTuner.X, spotTuner.y.reshape(-1, 1)), axis=1), columns=spotTuner.var_name + ["y"])
    df['l1'] = df['l1'].apply(lambda x: int(2**x))
    df['lr_mult'] = df['lr_mult'].apply(lambda x: x*1e-3)
    df['batch_size'] = df['batch_size'].apply(lambda x: int(2**x))
    df['epochs'] = df['epochs'].apply(lambda x: [5, 10, 15, 20][int(x)])
    df = df.rename(columns={'l1': 'config.l1', 
                            'lr_mult': 'config.learning_rate',
                            'batch_size': 'config.batch_size',
                            'epochs': 'config.epochs',
                            'y': 'mean_val_loss'})
    print(f"----------------------------\nFound {len(df)} trials.")
    return df

def bestConfig(df):
    df_tmp = df.groupby('trial_id').max('training_iteration').reset_index()
    df_tmp = df_tmp[df_tmp['mean_val_loss'] == df_tmp['mean_val_loss'].min()]
    best_config = df_tmp[['mean_val_loss', 'mean_accuracy', 'config.l1', 'config.batch_size', 'config.epochs', 'config.learning_rate']].to_dict(orient='records')[0]
    # for key, value in best_config.items():
    #     print(f'{key}: {value}') 
    return best_config

def bestConfigSpot(spotTuner):
    results = spotTuner.print_results(print_screen=False)
    best_config = {
        'mean_val_loss': None,
        'mean_accuracy': None,
        'config.l1': None,
        'config.batch_size': None,
        'config.epochs': None,
        'config.learning_rate': None,
    }
    for conf in results:
        if conf[0] == 'l1':
            best_config['config.l1'] = int(2**conf[1])
        elif conf[0] == 'batch_size':
            best_config['config.batch_size'] = int(2**conf[1])
        elif conf[0] == 'epochs':
            best_config['config.epochs'] = [5, 10, 15, 20][int(conf[1])]
        elif conf[0] == 'lr_mult':
            best_config['config.learning_rate'] = conf[1] * 1e-3
    return best_config

def repeatedTraining(config, iter, file_name):
    res = []
    for i in range(iter):
        torch.manual_seed(i)
        random.seed(i)
        np.random.seed(i)
        score = {'name': file_name}
        net = train_fashion_mnist(config)
        class_acc, acc = test_class_accuracy(net)
        for val, name in zip(class_acc,class_names):
            score[name] = val
        score['overall'] = acc
        res.append(score)
        df = pd.DataFrame(res)
        df.to_csv(f'./results/evaluation_data/{file_name}.csv')
    return df

In [14]:
## Load Data
df_e6 = pd.read_csv(f'{eval_data_path}/tuning_logs/e6-1.csv', index_col=0)
df_e6[['config.learning_rate']] = df_e6[['config.learning_rate']].astype(float)
idx = df_e6.groupby('trial_id')['training_iteration'].idxmax()
df_flat_e6 = df_e6.loc[idx,]

df_e7 = pd.read_csv(f'{eval_data_path}/tuning_logs/e7-1.csv', index_col=0)
df_e7[['config.learning_rate']] = df_e7[['config.learning_rate']].astype(float)
idx = df_e7.groupby('trial_id')['training_iteration'].idxmax()
df_flat_e7 = df_e7.loc[idx,]

spotTuner_e8 = experimentPathToSpotTuner("experiment_data/e8_spotPython/spot_runs_30-08-2023.pkl")
df_flat_e8 = pd.read_csv(f'{eval_data_path}/tuning_logs/e8.csv', index_col=0)

In [15]:
default_config = {
    'mean_val_loss': None,
    'mean_accuracy': None,
    'config.l1': 64,
    'config.batch_size': 64,
    'config.epochs': 15,
    'config.learning_rate': 0.001,
}

configs = [default_config,
           bestConfig(df_e6),
           bestConfig(df_e7),
           bestConfigSpot(spotTuner_e8)]

df = pd.DataFrame(configs)[['config.l1', 'config.batch_size', 'config.epochs', 'config.learning_rate']]
df.index = ['Default', 'Hyperband', 'Random Search', 'SMBO']
df['trials'] = ['-', f'{len(df_flat_e6)} (~6h)', f'{len(df_flat_e7)} (~6h)', f'{len(df_flat_e8)} (~6h)']
df = df.sort_values(by=['config.l1','config.learning_rate'])
df.columns = ['L1 Units', 'Batch Size', 'Epochs', 'Learning Rate', '# Trials']
df.to_csv(f'{eval_data_path}/best_config.csv', index=True)

<span style="color: #ff6900; font-size: 48px; font-weight: bold;">Evaluation</span>
<p style="font-size: 16px;"><i>Verification of Predictive Accuracy and Coverage of the Use Case</i></p>

<header>
    <span style="color: #ff6900; font-size: 30px; font-weight: bold;">Tuning Results</span>
</header>

**🏆Best Configurations**
- more _Units_,
- smaller _Batch Size_,
- not a lot of _Epochs_ needed since ...
- ... fairly high _Learning Rate_!

In [17]:
pd.read_csv(f'{eval_data_path}/best_config.csv', index_col=0)

,L1 Units,Batch Size,Epochs,Learning Rate,# Trials
Default,64,64,15,0.001000,-
Random Search,128,16,10,0.054035,87 (~6h)
Hyperband,128,16,15,0.098538,340 (~6h)
SMBO,256,32,10,0.091179,68 (~6h)


**#️⃣of Trials - Why is that so different?**
- Hyperband focuses resources on promising configurations by combining random sampling and early elimination of poor configurations 

In [70]:
e7_grouped = df_e7.groupby('config.epochs')
df7 = e7_grouped.get_group(20)

e6_grouped = df_e6.groupby('config.epochs')
df6 = e6_grouped.get_group(20)

fig = make_subplots(rows=1, cols=2, subplot_titles=["Random Search", "Hyperband"])

for trial in df7['trial_id'].unique().tolist():
    df = df7[df7['trial_id'] == trial]
    fig.add_trace(go.Scatter(x=df['training_iteration'], y=df['mean_val_loss'], mode='lines'), row=1, col=1)
fig.update_xaxes(title_text="Epoch", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=1)

for trial in df6['trial_id'].unique().tolist():
    df = df6[df6['trial_id'] == trial]
    fig.add_trace(go.Scatter(x=df['training_iteration'], y=df['mean_val_loss'], mode='lines'), row=1, col=2)
fig.update_xaxes(title_text="Epoch", row=1, col=2)
fig.update_yaxes(title_text="Loss", row=1, col=2)

fig.update_layout(title_text="Training Loss Comparison", showlegend=False)

fig.show()

<header>
    <span style="color: #ff6900; font-size: 30px; font-weight: bold;">Hyperparameter Interdependencies</span>
</header>

**🧪Tested Configurations Overview**

**🧪Tested Configurations Overview**

In [84]:
fig = px.parallel_coordinates(df_flat_e7, color="mean_val_loss",
                              title='Random Search Trial Configurations',
                              dimensions=['config.l1', 'config.batch_size', 'config.learning_rate',
                                          'config.epochs'], width=900)
fig.show()

**🧪Tested Configurations Overview**

In [85]:
fig = px.parallel_coordinates(df_flat_e6, color="mean_val_loss",
                              title='Hyperband Trial Configurations',
                              dimensions=['config.l1', 'config.batch_size', 'config.learning_rate',
                                          'config.epochs'], width=900)
fig.show()

**🧪Tested Configurations Overview**

In [86]:
fig = px.parallel_coordinates(df_flat_e8, color="mean_val_loss",
                              title='SMBO Trial Configurations',
                              dimensions=['config.l1', 'config.batch_size', 'config.learning_rate',
                                          'config.epochs'], width=900)
fig.show()